In [ ]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2
import os

from files import ImageFile
from processor import (GrayScaleProcessor, BlurProcessor, SobelProcessor, 
                       ThresholdProcessor, MorphologyProcessor, HistogramEqualizationProcessor,
                      CannyProcessor, ScaleProcessor, HoughLinesProcessor,
                      SaveImageProcessor, InvertProcessor, RetrieveImageProcessor)
from segment_processor import (HierarchyContourSegmenter, LargeSegmentFilter, 
                               SmallSegmentFilter, LargeAreaSegmentFilter, ContainedSegmentFilter, UniqueSegmentFilter,
                              RatioSegmentFilter)
from feature_extractor import (SimpleFeatureExtractor, ThreeChannelsFeatureExtractor)
from segment_classifier import ManualSegmentClassifier, DeepCnnSegmentClassifier
from processor_stack import ProcessorStack
import cv2
import itertools
import numpy as np

#test_image= ImageFile('PlateCar')
#test_image= ImageFile('Brad meter 1')
#test_image= ImageFile('Brad meter 2')
#test_image= ImageFile('Davide take 1')
#test_image= ImageFile('Davide take 2')
#test_image= ImageFile('Erik') #Hard set adaptiveBlockSize=9 to catch digits
#test_image= ImageFile('Gro') #Impossible electric2.jpg for erode 5
#test_image= ImageFile('Hans Martin') #quite Hard
#test_image= ImageFile('Knut G')
#test_image= ImageFile('Ole auto')
#test_image= ImageFile('Ole blitz ikke')
#test_image= ImageFile('Ole blitz')
#test_image= ImageFile('Ole liggende')
#test_image= ImageFile('Roy') #small picture, need to scale
#test_image= ImageFile('Sven Erik close up')
#test_image= ImageFile('Sven Erik')
#test_image= ImageFile('Thomas') #Digital

#test_image= ImageFile('downloaded/139889743') 
#test_image= ImageFile('downloaded/LeckyMeterDavidSillitoe460') 

#test_image= ImageFile('downloaded/metric meter') 

#test_image= ImageFile('downloaded/Electric-Meter') 
def manual_classify(image):    
    adaptiveBlockSizes = [9, 17, 31]
    erodes = [(3, 3), (5, 5), (7, 7)]
    dilates = [(3, 3), (5, 3), (7, 3)]
    segments = np.empty( shape=(0, 0) )

    for adaptiveBlockSize, (erode_x, erode_y), (dilate_x, dilate_y)  in itertools.product(adaptiveBlockSizes, erodes, dilates):
        digitDetectorStack = [ScaleProcessor(maxHeight=800, maxWidth=800),
             SaveImageProcessor(),
             GrayScaleProcessor(),
             BlurProcessor(blur_x=5, blur_y=5),
             HistogramEqualizationProcessor(HistogramEqualizationProcessor.CLAHE),
             ThresholdProcessor(ThresholdProcessor.ADAPTIVE, adaptiveBlockSize=adaptiveBlockSize), # 31 for normal. Change this to catch digits!
             MorphologyProcessor(shape=cv2.MORPH_RECT, ksize=(erode_x, erode_y), op = MorphologyProcessor.ERODE), #Change this
             MorphologyProcessor(shape=cv2.MORPH_RECT, ksize=(dilate_x, dilate_y), op = MorphologyProcessor.DILATE),
             HierarchyContourSegmenter(),
             RetrieveImageProcessor(),
             LargeSegmentFilter(min_width=4, min_height=13),
             SmallSegmentFilter(max_width=100, max_height=200),
             LargeAreaSegmentFilter(min_area = 120),
             RatioSegmentFilter(min_h_w_ratio = 1, max_h_w_ratio = 4)
             #ContainedSegmentFilter()
        ] 
        digitDetector = ProcessorStack(digitDetectorStack)
        digitDetector.process(image)
        output = digitDetector.get_output()
        if len(segments) == 0:
            segments = output['segments']
            image = output['originalImg']
        else:
            segments = np.concatenate((segments, output['segments']), axis = 0)
        #digitDetector.display()
    segmentFilterStack = [UniqueSegmentFilter(threshold = 0.6),
                          ThreeChannelsFeatureExtractor(feature_size=32, extension = 0, method=ThreeChannelsFeatureExtractor.PAD_BACKGROUND),
                          ManualSegmentClassifier()
                          ]
    segmentFilter = ProcessorStack(segmentFilterStack)
    segmentFilter.process(image, segments = segments)
    regions = segmentFilter.get_output()['regions']
    segment_types = segmentFilter.get_output()['segment_types']
    return (regions, segment_types)

image = cv2.imread("data/Erik.jpg")
image = cv2.imread("Fail Images/201612-1454432228873.jpg")

manual_classify(image)

continue_from_file = 'Erik.jpg'
is_processed = True
for root, dirs, files in os.walk("data"):
    for file in files:
        if file.lower().endswith(continue_from_file):
            is_processed = False
        if not is_processed:
            if file.lower().endswith(('.png','.tif','.jpg', '.jpeg')):
                image = cv2.imread(os.path.join(root, file))
                #cv2.imshow("original", image, cv2.WINDOW_NORMAL)
                print('Classifying file: ', os.path.join(root, file))
                regions, labels = manual_classify(image)
                np.savez(os.path.join("classified", file), regions=regions, labels = labels)

#np.savez("Davide_1", regions=regions, labels = segmentFilter.get_output()['segment_types'])

#segmentFilter.display()
#digitDetector.display()

In [ ]:
#Concat all classified regions
X_train = np.empty( shape=(0, 3, 32, 32) )
y_train = np.empty( shape=(0,) )
for root, dirs, files in os.walk("classified"):
    for file in files:
        if file.lower().endswith(('.npz')):
            with np.load(os.path.join(root, file)) as f:
                regions = f['regions']
                regions = np.rollaxis(regions, 3, 1)
                labels = f['labels']
                if len(X_train) == 0:
                    X_train = regions
                    y_train = labels
                else:
                    X_train = np.concatenate((X_train, regions), axis = 0)
                    y_train = np.concatenate((y_train, labels), axis = 0)
                    
np.savez(os.path.join("classified", "total"), X_train=X_train, y_train = y_train)

In [ ]:
#Add flipping and rotating images for non-digit regions
import numpy as np
import os
import cv2
with np.load(os.path.join("classified", "total.npz")) as f:
    X_train = f['X_train']
    y_train = f['y_train']

non_digits = X_train[y_train == 10]
non_digits = np.rollaxis(non_digits, 2, 1)
non_digits = np.rollaxis(non_digits, 3, 2)
#new_non_digits = np.empty(shape=(0, 32, 32, 3))
w, h = 32, 32
M1 = cv2.getRotationMatrix2D((w/2, h/2), 90, 1.0)
M2 = cv2.getRotationMatrix2D((w/2, h/2), 270, 1.0)

new_non_digits = []

for non_digit in non_digits:
    new_non_digits.append(cv2.flip(non_digit, 1))
    new_non_digits.append(cv2.flip(non_digit, -1))
    new_non_digits.append(cv2.flip(non_digit, 0))
    new_non_digits.append(cv2.warpAffine(non_digit, M1, (w, h)))
    new_non_digits.append(cv2.warpAffine(non_digit, M2, (w, h)))
    
new_non_digits = np.asarray(new_non_digits)
print new_non_digits.shape
new_non_digits = np.rollaxis(new_non_digits, 3, 1)
new_y_non_digits = np.empty(shape=(len(new_non_digits),), dtype = int)
new_y_non_digits.fill(10)

X_train = np.concatenate((X_train, new_non_digits), axis = 0)
y_train = np.concatenate((y_train, new_y_non_digits), axis = 0)
np.savez(os.path.join("classified", "total_add_rotation"), X_train=X_train, y_train = y_train)

In [ ]:
print X_train.shape
print y_train.shape
print y_train.dtype

In [ ]:
#Plot some random images from classified regions
%matplotlib inline

import random
import matplotlib.pyplot as plt
import matplotlib.cm as cm
def plot_image(img):
    x = np.rollaxis(img, 0, 3)
    if (x.shape[2] == 1):
        x = np.squeeze(x, axis=(2,))
        plt.imshow(x, cmap = cm.Greys_r)
    else:
        plt.imshow(x)
    plt.show()

def plot_random_image(imgs, target=None, n=10):
    indices = random.sample(xrange(0, imgs.shape[0]),  n)
    for index in indices:
        plot_image(imgs[index, :, :, :])
        if target != None:
            print target[index]

#plot_random_image(X_train[y_train!=10], y_train[y_train!=10],n = 100 )
plot_random_image(new_non_digits, new_y_non_digits,n = 100 )


In [ ]:
import numpy as np
segments = np.asarray([[1,2,3,4], [4,3,2,1], [4,5,4,3]])
a = segments[np.asarray([False, False, False])]

In [ ]:
len(a)